LSTM 

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(49391, 300, weights=[embedding_matrix])(inp)
x = SpatialDropout1D(S_DROPOUT)(x)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
x = Dense(16, activation="relu")(conc)
x = Dropout(DROPOUT)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))
pred_val_lstm_y = model.predict([val_X], batch_size=1024, verbose=1)
pred_test_lstm_y = model.predict([test_X], batch_size=1024, verbose=1)
del all_embs, model, inp, x
import gc; gc.collect()
time.sleep(10)

CNN

In [ ]:
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Conv2D
import tensorflow as tf
from keras.layers import concatenate
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(34176, embed_size, weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(tf.keras.layers.MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = tf.keras.layers.Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(5, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = model_cnn(embedding_matrix)
model.summary()
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model

from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from IPython.core.display import Image
keras.utils.plot_model(model, "Document_Classification_Model.png", show_shapes=True)

model.fit(train_X, train_y, batch_size=512, epochs=4, validation_data=(val_X, val_y))
pred_val_cnn_y = model.predict([val_X], batch_size=1024, verbose=1)
pred_test_cnn_y = model.predict([test_X], batch_size=1024, verbose=1)
pred_test_cnn_y1 = np.round(pred_test_cnn_y,4)

# del word_index, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

Mix two models

In [ ]:
pred_val_y = 0.6 * pred_val_lstm_y + 0.4 * pred_val_cnn_y  # two random numbers :)
pred_test_y = 0.6 * pred_test_lstm_y + 0.4 * pred_test_cnn_y 
sample1  = pd.read_csv("C:\\Users\\82107\\Downloads\\sample_submission.csv", index_col=0)
sample1[sample1.columns] = pred_test_y
sample1.to_csv("C:\\Users\\82107\\Downloads\\submission3.csv")